### YOLOv8 demo for CAST

<b>Run block for required dependencies:</b>

In [ ]:
# use this block to download requirements into virtual environment
%pip install -r requirements.txt

<u>Import statements<u>

In [5]:
import os
from ultralytics import YOLO
import cv2
import torch
import requests
import zipfile
import shutil

<u>Importing training dataset for demo</u>

For this demo, we will be training a YOLOv8 on African wildlife imagery pulled from https://ultralytics.com/assets/african-wildlife.zip. From ultralytics, "This dataset showcases four common animal classes typically found in South African nature reserves. It includes images of African wildlife such as buffalo, elephant, rhino, and zebra, providing valuable insights into their characteristics. Essential for training computer vision algorithms, this dataset aids in identifying animals in various habitats, from zoos to forests, and supports wildlife research.... 

The African wildlife objects detection dataset is split into three subsets:

Training set: Contains 1052 images, each with corresponding annotations.
Validation set: Includes 225 images, each with paired annotations.
Testing set: Comprises 227 images, each with paired annotations."

In [7]:
# using requests, download zip file and extract contents

# url of zipfile
url_of_zip = 'https://ultralytics.com/assets/african-wildlife.zip'

# directory to be saved into
wd = os.getcwd()
zip_directory = os.path.join(wd, 'animals.zip')

# download request
response = requests.get(url_of_zip)
with open(zip_directory, 'wb') as f:
    f.write(response.content)

# use the working directory as the extraction target
extract_to = wd  # Since we're extracting to the working directory

# extract the zip file
with zipfile.ZipFile(zip_directory, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# list extracted files
print(f'Extracted files: {os.listdir(extract_to)}' )


Extracted files: ['valid', 'video_test.py', 'unneeded_files', '.DS_Store', 'requirements.txt', 'test', 'experiments', 'hpc_other', 'YOLO_demo.ipynb', 'car_detector.ipynb', 'README.md', 'image-toolkit.py', '.gitattributes', 'train', 'slurm.bash', 'animals.zip', '.git', 'upload']


<b>Clear directory of training files if needed:</b>

In [8]:
# initialize file name
file_name = 'animals.zip'

# construct the file path
file_path = os.path.join(os.getcwd(), file_name)

# delete the zip file
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"The file {file_name} has been deleted.")
else:
    print(f"The file {file_name} does not exist.")

# list of directory paths you want to delete
directories = ["./train", "./test", "./valid"]

# loop through each directory in the list
for directory_path in directories:

    # check if the directory exists
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        
        # use shutil.rmtree() to delete the directory
        shutil.rmtree(directory_path)
        print(f"The directory {directory_path} has been deleted.")

    else:
        print(f"The directory {directory_path} does not exist.")


The file animals.zip has been deleted.
The directory ./train has been deleted.
The directory ./test has been deleted.
The directory ./valid has been deleted.
